In [27]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
np.random.seed(42)
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
df = pd.read_csv('../../data/judge-1377884607_tweet_product_company.csv',encoding='unicode-escape')
df.sample(8)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
5558,RT @mention best thing abt @mention for #SXSW?...,NaN,No emotion toward brand or product
8350,Gary is starting a new show &quot;Daily Grape&...,NaN,No emotion toward brand or product
2303,"The walk by Lady Bird Lake was lovely, but Goo...",Other Google product or service,Negative emotion
927,C34: ItÛªs Not a Retreat! {link} [codes valid...,NaN,No emotion toward brand or product
1963,Holler Gram for iPad on the iTunes App Store: ...,iPad or iPhone App,Positive emotion
1672,"I asked, &quot;Why should(n't) I get an iPad 2...",NaN,No emotion toward brand or product
4475,"LOL, I don't see a single PC user around meÛÓ...",Apple,Positive emotion
4262,"Video: Tim Wu explains to me #netneutrality, i...",NaN,No emotion toward brand or product


In [29]:
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
sw = stopwords.words('english')

In [30]:
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [31]:
def doc_preparer(doc, stop_words=sw):
    '''
    
    :param doc: a document from the corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in stop_words]
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

In [32]:
regex_token

NameError: name 'regex_token' is not defined

In [33]:
X = df.tweet_text.astype('str')
y = df.is_there_an_emotion_directed_at_a_brand_or_product

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.25)

In [35]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adamroth/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adamroth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
token_docs = [doc_preparer(doc, sw) for doc in X_train]
token_docs[0:5]

['good design mean get simple mention talk text way impact healthcare africa forget iphone amhealth sxsw',
 'wish sxsw see rumor demo mention new social network circle link via mention',
 'rt mention problem time stop link edchat musedchat sxsw sxswi classical newtwitter',
 'mention new ipad apps speech therapy communication showcased sxsw conference link augcomm hcsm mhealth',
 'ipad queue epic sxsw link']

In [10]:
X_t, X_val, y_t, y_val = train_test_split(token_docs, y_train, test_size=.25, random_state=42)

In [11]:
cv = CountVectorizer()

X_t_vec = cv.fit_transform(X_t)
X_t_vec  = pd.DataFrame.sparse.from_spmatrix(X_t_vec)
X_t_vec.columns = sorted(cv.vocabulary_)
X_t_vec.set_index(y_t.index, inplace=True)

In [12]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()

mnb.fit(X_t_vec, y_t)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
prior_1 = y_t.value_counts()['Positive emotion']/len(y_t)
prior_0 = y_t.value_counts()['Negative emotion']/len(y_t)
prior_2 = y_t.value_counts()['No emotion toward brand or product']/len(y_t)
prior_3 = y_t.value_counts()['I can\'t tell']/len(y_t)
print(prior_0, prior_1, prior_2, prior_3)
print(np.log(prior_1))


0.06257332811888933 0.3318341806804849 0.5879937426671881 0.01759874853343762
-1.1031198906169057


In [14]:
mnb.class_log_prior_

array([-4.03992749, -2.77141616, -0.53103897, -1.10311989])

In [15]:
cv = CountVectorizer()
X_t_vec = cv.fit_transform(X_t)
X_t_vec  = pd.DataFrame.sparse.from_spmatrix(X_t_vec)
X_t_vec.columns = sorted(cv.vocabulary_)
X_t_vec.set_index(y_t.index, inplace=True)


X_val_vec = cv.transform(X_val)
X_val_vec  = pd.DataFrame.sparse.from_spmatrix(X_val_vec)
X_val_vec.columns = sorted(cv.vocabulary_)
X_val_vec.set_index(y_val.index, inplace=True)
cv.vocabulary_

{'wave': 5689,
 'buzz': 733,
 'rt': 4434,
 'mention': 3244,
 'interrupt': 2670,
 'regularly': 4273,
 'schedule': 4503,
 'sxsw': 5064,
 'geek': 2107,
 'program': 4062,
 'big': 536,
 'news': 3481,
 'link': 3009,
 'google': 2187,
 'circle': 912,
 'ipad': 2695,
 'sell': 4558,
 'retail': 4345,
 'store': 4948,
 'except': 1763,
 'pop': 3947,
 'austin': 348,
 'igottagetit': 2528,
 'way': 5691,
 'matt': 3199,
 'fricking': 2029,
 'damon': 1247,
 'party': 3751,
 'maggie': 3122,
 'mae': 3118,
 'part': 3743,
 'presentation': 4015,
 'good': 2182,
 'thing': 5228,
 'crash': 1167,
 'instead': 2644,
 'work': 5812,
 'best': 519,
 'iphone': 2700,
 'android': 193,
 'launch': 2921,
 'major': 3132,
 'new': 3475,
 'social': 4778,
 'network': 3469,
 'call': 751,
 'possibly': 3969,
 'today': 5292,
 'via': 5598,
 'sxswi': 5081,
 'sxswparty': 5092,
 'upstairs': 5530,
 'maes': 3119,
 'th': 5199,
 'st': 4894,
 'temp': 5174,
 'amp': 182,
 'congress': 1056,
 'attendee': 324,
 'excuse': 1770,
 'walk': 5662,
 'around':

In [16]:
mnb = MultinomialNB()

mnb.fit(X_t_vec, y_t)
y_hat = mnb.predict(X_val_vec)
print(accuracy_score(y_val, y_hat))
print(confusion_matrix(y_val, y_hat))

0.6539589442815249
[[  0   0  19   8]
 [  0  17  45  44]
 [  0   5 817 210]
 [  0   1 258 281]]


In [17]:
y_val.value_counts()

No emotion toward brand or product    1032
Positive emotion                       540
Negative emotion                       106
I can't tell                            27
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [18]:
confusion_matrix(y_val,y_hat)

array([[  0,   0,  19,   8],
       [  0,  17,  45,  44],
       [  0,   5, 817, 210],
       [  0,   1, 258, 281]])

In [19]:
#Not great, but not bad for a first few models

In [20]:
type(cv.vocabulary_)

dict

### Visualization of word counts

In [21]:
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

In [37]:
fdist = FreqDist(cv.vocabulary_)
plt.figure(figsize=(10,10))
fdist.plot(30)

AttributeError: 'dict' object has no attribute 'ascending'

,aapl,aaron,aarpbulletin,ab,abandon,abc,ability,able,abnormal,abound,...,zip,zlf,zms,zomb,zombie,zomg,zone,zoom,zuckerberg,zynga
9089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
806,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4543,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
